# Model Management

This notebook shows how you can manage the installed models in BucketFS.

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI Lab](../main_config.ipynb).
2. [Initialize the Transformer Extension](te_init.ipynb).

## Setup

### Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Let's bring up JupySQL and connect to the database via SQLAlchemy. 
Please refer to the documentation of [sqlalchemy-exasol](https://github.com/exasol/sqlalchemy-exasol) for details on how to connect to the database using the Exasol SQLAlchemy driver.

In [ ]:
%run ../utils/jupysql_init.ipynb

## Install a model

In this example we need some model to demonstrate the available administration UDFs. For this we install the [Ekman emotions classifier](https://huggingface.co/arpanghoshal/EkmanClassifier) model.

We need to load the model from the Hugging Face Hub into the [BucketFS](https://docs.exasol.com/db/latest/database_concepts/bucketfs/bucketfs.htm). This could potentially be a long process. Unfortunately, we cannot tell exactly when it has finished. The notebook's hourglass may not be a reliable indicator. BucketFS will still be doing some work when the call issued by the notebook returns. Please wait for a few moments after that, before querying the model.

You might see a warning that some weights are newly initialized and the model should be trained on a down-stream task. Please ignore this warning. For the purpose of this demonstration, it is not important.

In [ ]:
from exasol.nb_connector.model_installation import install_model, TransformerModel
from transformers import AutoModelForSequenceClassification
# This is the name of the model at the Hugging Face Hub
MODEL_NAME = 'arpanghoshal/EkmanClassifier'
install_model(ai_lab_config, TransformerModel(MODEL_NAME, 'sequence_classification', AutoModelForSequenceClassification))

## List available models

Now we have at least one model installed in BucketFS. (If you have already executed other notebooks from the Transformer Extension tuturial, you probably will have more models installed)

We now can use the `TE_LIST_MODELS_UDF` UDF to check all available models.

In [ ]:
%%sql --save udf_output
SELECT TE_LIST_MODELS_UDF(
    '{{ai_lab_config.bfs_connection_name}}',
    '{{ai_lab_config.bfs_model_subdir}}'
)

## Delete a model

You can delete a model using the `TE_DELETE_MODEL_UDF` UDF.
In the following example we delete the model, which was installed before.
Note: We don't use the `%%sql` expression here because it interfers with the test tools. But feel free to use `%%sql` in your own notebooks.


In [ ]:
model_delete = %sql SELECT TE_DELETE_MODEL_UDF( \
    '{{ai_lab_config.bfs_connection_name}}', \
    '{{ai_lab_config.bfs_model_subdir}}', \
    'arpanghoshal/EkmanClassifier', \
    'text-classification' \
)
model_delete

Running the `TE_LIST_MODELS_UDF` again now will show that the model was removed from BucketFS.

In [ ]:
%%sql --save udf_output
SELECT TE_LIST_MODELS_UDF(
    '{{ai_lab_config.bfs_connection_name}}',
    '{{ai_lab_config.bfs_model_subdir}}'
)

`TE_DELETE_MODEL_UDF` will not fail if an invalid parameter was given, instead it will indicate an error in the output columns `success` and `error_message`.
We can observe this behaviour if we now try to delete the same model again.

Note: We don't use the `%%sql` expression here because it interferes with the test tools. But feel free to use `%%sql` in your own notebooks.


In [ ]:
model_delete_with_error = %sql SELECT TE_DELETE_MODEL_UDF( \
    '{{ai_lab_config.bfs_connection_name}}', \
    '{{ai_lab_config.bfs_model_subdir}}', \
    'arpanghoshal/EkmanClassifier', \
    'text-classification' \
)
model_delete_with_error